# Demo 2 - Sentiment Analysis using CNN + Hyperparameter Tuning

##**Scenario**
You're building a sentiment analysis model to classify movie reviews as positive or negative using the IMDB dataset. You'll apply regularization, gradient clipping, and hyperparameter tuning using Keras Tuner, and visualize the training process with TensorBoard.

## **Objective**
* Use a CNN-based architecture for text classification

* Tune hyperparameters like embedding size, filters, kernel size, dropout, learning rate

* Apply gradient clipping and dropout
* Visualize training using TensorBoard

* Evaluate model on test data


## Step 1: Import Required Libraries
Import all required libraries including Keras Tuner and preprocessing tools.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras_tuner.tuners import RandomSearch # This line should now work after installing the package.

## Step 2: Load and Preprocess IMDB Dataset
Load movie reviews, tokenize them, and pad to uniform sequence length.

In [ ]:
vocab_size = 10000
maxlen = 200

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

## Step 3: Define 1D CNN Model for Hyperparameter Tuning
Defines a tunable 1D CNN model using embedding, convolution, pooling, dropout, and sigmoid output.


In [ ]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
                               output_dim=hp.Int('embed_dim', 32, 128, step=32),
                               input_length=maxlen))
    model.add(layers.Conv1D(filters=hp.Int('filters', 32, 128, step=32),
                            kernel_size=hp.Choice('kernel_size', [3, 5]),
                            activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dropout(hp.Choice('dropout', [0.2, 0.3, 0.5])))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4]),
            clipnorm=1.0  # Gradient clipping
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

## Step 4: Set Up TensorBoard and EarlyStopping Callbacks
Enables training monitoring and stops training early if validation doesn’t improve.

In [ ]:
log_dir = "logs/demo2"
tensorboard_cb = callbacks.TensorBoard(log_dir=log_dir)
early_stop_cb = callbacks.EarlyStopping(patience=3, restore_best_weights=True)

## Step 5: Run Random Search with Keras Tuner
Automatically tests multiple hyperparameter combinations and logs results.

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='sentiment_analysis'
)

tuner.search(x_train, y_train, epochs=5, validation_split=0.2,
             callbacks=[tensorboard_cb, early_stop_cb], batch_size=64)

## Step 6: Retrieve the Best Model and Evaluate
Fetch and evaluate the best model found during tuning.


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

## Step 7: Launch TensorBoard in Google Colab
Launches TensorBoard in notebook to view training curves, histograms, and scalar logs.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/demo2